# Часть 1. 

# Начало работы с данными.

## Первый взгляд на рабочие данные 

Загрузка рабочего датасета:

In [1]:
import pandas as pd

hotels = pd.read_csv('hotels.csv')
hotels.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


### Базовые характеристики рабочего датасета:

In [2]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

### Характеристики рабочего датасета, представляющие интерес:

- Датасет содержит 387 тысяч строк (индивидуальных отзывов к отелям)

- Каждая строка содержит информацию о 17 признаках, 10 из которых (включая целевой признак reviewer_score) 
  являются количественными, 6 - качественными, а один признак представляет собой дату в строковом виде (object)

- Из 10 количественных признаков 5 представлены в виде целых чисел (int64), 4 - в виде
 чисел с плав. запятой (float64), а один признак (days_since_review) - в строковом виде (object) 

- Пропуски в данных имеются только в двух признаках - географических координат отеля lat и lng.
 Оба этих признака содержат абсолютно одинаковое количество пропусков (скорее всего, находящихся в одних 
 и тех же строках для каждого признака)

- Признаки negative_review и positive_review содержат в себе собственно текст отзывов и требуют проведения
 хотя бы простейшего парсинга текста для получения из них полезной информации
 
- Признак tags должен представлять из себя список тегов; для получения из него полезной информации
 требутся провести специализированный парсинг

## Разбиение и базовая предобработка исходного датасета.

Разбиваем датасет на X и y. Все дальнейшие преобразования будут осуществляться с датасетом X, а не с исходным датасетом.

In [26]:
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

Для удобства обработки признак days_since_review должен быть сразу же переведён из строкового в числовой формат,
а review_date - в формат datetime.

In [27]:
X['review_date'] = pd.to_datetime(X['review_date'], format='%m/%d/%Y')
X['days_since_review'] = X['days_since_review'].apply(lambda x: int(x[:-4]))

### Устранение пропусков в признаках lat и lng

In [5]:
missing_num = sum(X['lat'].isna())

print('Процент пропусков: ', str(round((missing_num/X.shape[0]) * 100, 1)), '%')

Процент пропусков: 0.6%


Так как пропуски наличествуют всего лишь в двух признаках, и при этом в обоих занимают менее процента от общего объёма данных в них,
в данном проекте не будут использоваться продвинутые методы заполнения пропусков в данных. Вместо этого пропущенные значения будут просто
заменены нулями.

In [28]:
X = X.fillna(0)

### Базовая обработка дат оставления отзывов

Посмотрим на диапазон дат, в которые были оставлены отзывы:

In [7]:
print(X['review_date'].min(), X['review_date'].max())

2015-08-04 00:00:00 2017-08-03 00:00:00


Видно, что все отзывы в датасете были оставлены на протяжении двух лет: с августа 2015 по август 2017. 
Из-за небольшого диапазона лет информация о годе оставления отзыва является малополезной (вряд ли типичный турист из 2015 г.
будет сколько-нибудь заметно отличаться в своих ожиданиях и предпочтениях от туриста из 2017 г.). С другой стороны, так же маловероятно,
что на настроение рецензета должен влиять точный день оставления отзыва.

Имеет смысл оставить только месяц оставления отзыва (в самом деле, предпочтения людей, путешествующих в разное время года, могут сильно отличаться), 
выделив его в отдельный признак, а от исходного признака даты избавиться на этапе финального удаления нечисловых признаков.

In [29]:
X['review_month'] = X['review_date'].apply(lambda x: x.month)

### Базовая обработка национальностей рецензентов (reviewer_nationality):

Потенциально важным признаком может оказаться национальность рецензента.
Полезно будет присмотреться к нему подробнее. Посмотрим на наиболее и наименее часто представленные в датасете национальности.

In [9]:
X['reviewer_nationality'].value_counts()

reviewer_nationality
 United Kingdom               184033
 United States of America      26541
 Australia                     16239
 Ireland                       11126
 United Arab Emirates           7617
                               ...  
 Cook Islands                      1
 Guinea                            1
 Comoros                           1
 Anguilla                          1
 Grenada                           1
Name: count, Length: 225, dtype: int64

Также присмотримся к синтаксису типичного значения этого признака:


In [10]:
X['reviewer_nationality'][0]

' United Kingdom '

Можно увидеть, что значения этого признака "обрамлены" пробелами с обеих сторон. 
Для удобства дальнейшей обработки от них можно сразу же избавиться:

In [30]:
X['reviewer_nationality'] = X['reviewer_nationality'].apply(lambda x: x[1:-1])
X['reviewer_nationality'][0]

'United Kingdom'

#### Свойства признака национальности рецензентов, представляющие интерес:

- В датасете представлены 225 национальностей со всего мира, включая малонаселённые отдалённые территории.

- Однако, около половины всех отзывов были оставлены британцами. Такая аномально большая большая пропорция, возможно, требует дальнейшего исследования.

На первой итерации обучения модели, однако, национальность рецензентов использоваться не будет - её предназначением будет оценить точность модели до проведения инженерии признаков.

**Внимание:** В данном проекте не будет производиться нормализация или стандартизация исходных данных, т.к. они не приносят полезного эффекта при использовании с алгоритмом случайного леса, который применяется в данном проекте как основа модели.

## Первая итерация обучения модели на обработанных данных

In [31]:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели   
from sklearn.model_selection import train_test_split #инструмент для разбивки 

In [32]:
#Удаляем нечисловые значения
X = X.drop(['hotel_address', 'hotel_name', 'reviewer_nationality', 'negative_review',
            'positive_review', 'tags', 'review_date'], axis=1)

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования используется 25% от исходного датасета.  
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train_1, y_train_1)  

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred_1 = regr.predict(X_test_1)

# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются 
print('MAPE:', metrics.mean_absolute_percentage_error(y_test_1, y_pred_1))

MAPE: 0.1375711583743227


In [35]:
print('Точность модели на текущей итерации:', (1 - 0.13757)*100, '%')

Точность модели на текущей итерации: 86.24300000000001 %


Полученное выше значение точности модели в дальнейшем будем считать базовым - сравнение с ним будет использоваться для оценки эффективности новых конструируемых признаков.

# Часть 2.

# Анализ и обработка текстов отзывов и наборов тегов, их сопровождающих. 

## Приведение признака tags к удобному для обработки виду

Сначала вернём в датасет X данные признаки:

In [38]:
X['tags'] = hotels['tags']
X['positive_review'] = hotels['positive_review']
X['negative_review'] = hotels['negative_review']

Синтаксис типичного значения признака tags:

In [12]:
X['tags'][0]

"[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']"

Значение представляет собой перечисление содержащихся в нём тегов через запятые, после каждой из которых стоит пробел.
В дополнение к этому каждый тег заключен в двойное обрамление - внешнее из апострофов и внутреннее из пробелов.
Вся конструкция заключена в квадратные скобки.

При преобразовании значений в стандартный Python-список оставляем в каждом теге только собственно текст тега.

In [39]:
def tag_parser(tag_string):
    tag_list = tag_string.split(sep=' \', \' ')
    tag_list[0] = tag_list[0][3:]
    tag_list[-1] = tag_list[-1][:-3]
    return tag_list

X['tags'] = X['tags'].apply(tag_parser)

## Получение списка самых популярных тегов среди всех рецензентов 

In [14]:
tag_count = dict()

def tag_counter(tag_list):
    global tag_count
    for tag in tag_list:
        if tag in tag_count.keys():
            tag_count[tag] = tag_count[tag] + 1
        else:
            tag_count[tag] = 1
    return None

X['tags'].apply(tag_counter)
tag_count = pd.Series(tag_count)
tag_count.sort_values(ascending=False)[:10]

Leisure trip                      313593
Submitted from a mobile device    230778
Couple                            189212
Stayed 1 night                    145373
Stayed 2 nights                   100263
Solo traveler                      81235
Stayed 3 nights                    72000
Business trip                      61989
Group                              49088
Family with young children         45836
dtype: int64

### Промежуточный вывод:

- Пять наиболее часто встречающихся тегов, также являющихся единственными тегами, присутствующими в датасете в количестве более ста тысяч - это 
 теги Leisure trip, Submitted from a mobile device, Couple, Stayed 1 night и Stayed 2 nights. 
 
 В целях экономии времени и вычислительных ресурсов будем в дальнейшем учитывать только эти пять тегов, а информацию об остальных отбросим. 

## Кодирование пятёрки самых популярных тегов с помощью One Hot Encoding

In [40]:
def tag_encoder(tag_list):
    if tag_checked in tag_list:
        return 1
    else:
        return 0
    
for tag in ['Leisure trip', 'Submitted from a mobile device',
                'Couple', 'Stayed 1 night', 'Stayed 2 nights']:
    tag_checked = tag
    X[tag]=X['tags'].apply(tag_encoder)

## Парсинг отзывов

Разобьём тексты отзывов на отдельные слова и посмотрим, какие слова чаще всего встречаются в позитивных отзывах, а какие - в негативных. 

In [41]:
def review_parser(text):
    result = []
    for word in text.split():
        if len(word)>3:#Необходимо, чтобы исключить бесполезные для нас артикли, предлоги и.т.п. из списка слов.
            result.append(word.lower())
    return result

X['positive_review'] = X['positive_review'].apply(review_parser)
X['negative_review'] = X['negative_review'].apply(review_parser)

Аналогично подсчёту, проводившемуся в списках тегов, посчитаем слова в текстах отзывов во всём датасете и посмотрим на десять самых 
часто встречающихся слов в хороших отзывах и десять самых часто встречающихся слов в отзывах плохих.

### Самые частотные слова в позитивных отзывах:

In [17]:
good_review_word_count = dict()

def good_review_word_counter(word_list):
    global good_review_word_count
    for word in word_list:
        if word in good_review_word_count.keys():
            good_review_word_count[word] = good_review_word_count[word] + 1
        else:
            good_review_word_count[word] = 1
    return None

X['positive_review'].apply(good_review_word_counter)
pd.Series(good_review_word_count).sort_values(ascending=False)[:10]

staff        145874
location     144568
very         144568
room         105728
hotel         94000
good          84480
great         79160
were          68308
friendly      63870
breakfast     63307
dtype: int64

### Самые частотные слова в негативных отзывах:

In [18]:
bad_review_word_count = dict()

def bad_review_word_counter(word_list):
    global bad_review_word_count
    for word in word_list:
        if word in bad_review_word_count.keys():
            bad_review_word_count[word] = bad_review_word_count[word] + 1
        else:
            bad_review_word_count[word] = 1
    return None

X['negative_review'].apply(bad_review_word_counter)
pd.Series(bad_review_word_count).sort_values(ascending=False)[:10]

room         132025
negative      97092
very          60354
hotel         55916
were          46215
that          44238
breakfast     43528
have          41060
with          38968
small         37372
dtype: int64

### Промежуточные выводы:

- Люди, оставивишие положительные отзывы, больше всего хвалят персонал (stuff), расположение отеля (location), номер (room) и завтрак (breakfast)

- Люди, оставившие отрицательные отзывы, более всего ругают номер (room) и завтрак (breakfast)

Переведём эту информацию в числовой формат, также используя One Hot Encoding.

In [42]:
def word_encoder(word_list):
    if word_checked in word_list:
        return 1
    else:
        return 0
    
for word in ['stuff', 'location', 'room', 'breakfast']:
    word_checked = word
    X['good_'+word]=X['positive_review'].apply(word_encoder)

for word in ['room', 'breakfast']:
    word_checked = word
    X['bad_'+word]=X['negative_review'].apply(word_encoder)

X.sample(3)

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,lat,lng,review_month,...,Submitted from a mobile device,Couple,Stayed 1 night,Stayed 2 nights,good_stuff,good_location,good_room,good_breakfast,bad_room,bad_breakfast
67821,398,8.7,18,3754,0,3,339,45.463068,9.197937,8,...,0,0,1,0,0,0,0,0,0,0
383411,256,7.8,26,882,13,5,228,51.493741,-0.244896,12,...,0,1,1,0,0,0,0,0,0,0
268473,1444,7.8,23,5726,21,1,483,51.493508,-0.183435,4,...,0,1,1,0,0,0,1,0,1,0


## Вторая итерация обучения модели 

In [45]:
X = X.drop(['tags', 'positive_review', 'negative_review'], axis=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.25, random_state=42)
      
regr.fit(X_train_2, y_train_2)  
 
y_pred_2 = regr.predict(X_test_2)

print('MAPE:', metrics.mean_absolute_percentage_error(y_test_2, y_pred_2))

MAPE: 0.1350856464140379


In [46]:
print('Точность модели на текущей итерации:', (1 - 0.13508)*100, '%')

Точность модели на текущей итерации: 86.492 %


Как можно видеть, информация о наличии в отзыве одного или нескольких популярных тегов, а также полученная при парсинге текстов самих отзывов, является полезной - включение её в модель приводит к суммарному повышению её точности почти на половину процента. 

# Часть 3.

# Кодирование национальности рецензентов.

Возвращаем в X колонку с национальностями рецензентов из исходного датасета.

In [47]:
X['reviewer_nationality'] = hotels['reviewer_nationality']
X['reviewer_nationality'] = X['reviewer_nationality'].apply(lambda x: x[1:-1])

## Получение и форматирование информации о населении стран из внешнего источника

В данном проекте будет использоваться числовая кодировка национальностей с помощью представления 
каждой страны или территории в виде численности её населения. Информация о численности населения стран и территорий возьмём из общедоступного источника.

In [48]:
# Общедоступный датасет, содержащий информацию о странах мира (разработчик - Bnokoro)
url = 'https://raw.githubusercontent.com/bnokoro/Data-Science/master/countries%20of%20the%20world.csv'

countries = pd.read_csv(url)
countries[countries['Country']=='Korea, South ']

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
110,"Korea, South",ASIA (EX. NEAR EAST),48846823,98480,"496,0","2,45",0,"7,05",17800.0,"97,9","486,1","17,18","1,95","80,87",3,10,"5,85","0,033","0,403","0,563"


Нам нужна только информация о названиях стран и их населении.
Выделим эти два признака в отдельный датасет.

In [49]:
populations = countries[['Country', 'Population']]
populations.sample(5)

,Country,Population
136,"Micronesia, Fed. St.",108004
66,Faroe Islands,47246
53,Czech Republic,10235455
45,"Congo, Dem. Rep.",62660551
122,Macau,453125


После каждого названия страны в populations['Country'] пристутствует лишний пробел. Удаляем его:

In [50]:
populations['Country'] = populations['Country'].apply(lambda x: x[:-1])
populations['Country'][0]

C:\Users\New\AppData\Local\Temp\ipykernel_7948\3431658343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populations['Country'] = populations['Country'].apply(lambda x: x[:-1])


'Afghanistan'

При объединении датасетов может возникнуть проблема, если в них используются разные названия для части государств.
Скорее всего, оба датасета используют самые общеупотребительные в английском языке варианты названий государств без аббревиации
(e.g. China вместо People's Republic of China или PRC), но для США варианты United States и United States of America общеприняты в примерно одинаковой мере, поэтому их нужно проверить. 

In [ ]:
populations[populations['Country']=='United States']

,Country,Population
214,United States,298444215


Populations использует вариант United States, но выше мы уже видели, что в X используется вариант United States of America.
Необходимо исправить это, приведя названия к одному виду: 

In [51]:
def usa_format_fixer(string):
    if string=='United States':
        return 'United States of America'
    else:
        return string
    
populations['Country'] = populations['Country'].apply(usa_format_fixer)
populations[populations['Country']=='United States of America']

C:\Users\New\AppData\Local\Temp\ipykernel_7948\650688980.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populations['Country'] = populations['Country'].apply(usa_format_fixer)


,Country,Population
214,United States of America,298444215


Аналогичная проблема может возникнуть и с Южной Кореей - в populations она представлена как Korea, South

In [ ]:
populations[populations['Country']=='Korea, South']

,Country,Population
110,"Korea, South",48846823


...а в X - как South Korea

In [53]:
X[X['reviewer_nationality']=='South Korea'].sample(1)

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,lat,lng,review_month,...,Couple,Stayed 1 night,Stayed 2 nights,good_stuff,good_location,good_room,good_breakfast,bad_room,bad_breakfast,reviewer_nationality
340889,25,9.2,0,283,45,1,75,48.875345,2.316716,5,...,0,0,1,0,0,0,0,0,0,South Korea


Исправим эту проблему аналогичным образом (заодно на всякий случай и проблему с Северной Кореей тоже - хотя туристы из Северной Кореи практически не существуют, перестраховаться не помешает)

In [54]:
def korea_format_fixer(string):
    if string=='Korea, South':
        return 'South Korea'
    elif string=='Korea, North':
        return 'North Korea'
    else:
        return string
    
populations['Country'] = populations['Country'].apply(korea_format_fixer)
populations[populations['Country']=='South Korea']

C:\Users\New\AppData\Local\Temp\ipykernel_7948\3699561890.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populations['Country'] = populations['Country'].apply(korea_format_fixer)


,Country,Population
110,South Korea,48846823


## Кодирование национальностей рецензентов численностью населения их родной страны/территории

In [55]:
X = X.merge(populations, left_on='reviewer_nationality', right_on='Country', how='left')

Заменяем название страны в reviewer_nationality на соответствующую ей популяцию, после чего удаляем сделавшие своё дело и 
ставшие бесполезными колонки Country  и Population.

In [56]:
X['reviewer_nationality'] = X['Population']
X = X.drop(['Country', 'Population'], axis=1)

Однако теперь в признаке reviewer_nationality появились пропущенные значения из-за того, что изначально в нём было больше уникальных значений, чем стран в populations. Скорее всего, это объясняется тем, что датасете от bnokoro не были учтены некоторые малонаселённые территории. Эти пропуски естественней всего заполнить маленьким значением численности населения, например 50000.

In [57]:
X['reviewer_nationality'] = X['reviewer_nationality'].fillna(50000)

Наконец, приведём значения reviewer_nationality к виду целых чисел для ускорения обработки.

In [58]:
X['reviewer_nationality'] = X['reviewer_nationality'].astype('int32')

## Третья итерация обучения модели 

In [59]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, test_size=0.25, random_state=42)

regr.fit(X_train_3, y_train_3)  
      
y_pred_3 = regr.predict(X_test_3)

print('MAPE:', metrics.mean_absolute_percentage_error(y_test_3, y_pred_3))

MAPE: 0.13444068468175463


In [60]:
print('Точность модели на текущей итерации:', (1 - 0.13444)*100, '%')

Точность модели на текущей итерации: 86.556 %


# Часть 4.

# Использование информации о геграфическом положении отеля

Сначала посмотрим, в каких странах находятся отели:

In [61]:
def uk_format_fixer(string):
    if string=='Kingdom':
        return 'United Kingdom'
    else:
        return string

X['hotel_country'] = hotels['hotel_address'].apply(lambda x: x.split()[-1])
X['hotel_country'] = X['hotel_country'].apply(uk_format_fixer)
X['hotel_country'].value_counts()

hotel_country
United Kingdom    196774
Spain              45132
France             44830
Netherlands        43006
Austria            29178
Italy              27883
Name: count, dtype: int64

Закодируем данные о стране, в которой находится отель аналогично данным о национальности рецензентов.

In [62]:
X = X.merge(populations, left_on='hotel_country', right_on='Country', how='left')
X['hotel_country'] = X['Population']
X = X.drop(['Country', 'Population'], axis=1)

## Четвёртая итерация обучения модели

In [63]:
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, test_size=0.25, random_state=42)

regr.fit(X_train_4, y_train_4)  
      
y_pred_4 = regr.predict(X_test_4)

print('MAPE:', metrics.mean_absolute_percentage_error(y_test_4, y_pred_4))

MAPE: 0.1343734310555643


In [64]:
print('Точность модели на текущей итерации:', (1 - 0.13437)*100, '%')

Точность модели на текущей итерации: 86.563 %


Точность модели на уровне 86.563 % является приемлемой для целей данного проекта. Проект можно считать успешно выполненным.